In [15]:
#Basic Packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import confusion_matrix
import squarify

#To find addresses
import reverse_geocoder as rg
from geopy.geocoders import Nominatim

#Plot Maps
import folium
from folium.plugins import HeatMap, MarkerCluster 

#Functions
import Functions

%load_ext autoreload
%autoreload 2

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")


#Para fazer as regras de associação

import mlxtend.fr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
individuals = pd.read_csv('customers_clustered.csv')
basket = pd.read_csv('Customer Basket Dataset.csv')

In [11]:
individuals.set_index('customer_id', inplace=True)

In [12]:
basket.set_index('invoice_id', )

,customer_id,invoice_id,list_of_goods
0,7980,6494405,"['fresh bread', 'cake', 'grated cheese', 'fren..."
1,9995,6855482,"['bramble', 'beer', 'white wine', 'almonds', '..."
2,10060,3614116,"['fromage blanc', 'olive oil', 'tomato sauce',..."
3,1461,6464656,"['pokemon shield', 'white wine', 'google table..."
4,7830,10330249,"['olive oil', 'deodorant', 'whole weat flour',..."
...,...,...,...
79995,360,11470275,"['tomatoes', 'ham', 'zucchini', 'frozen vegeta..."
79996,3116,7483644,"['pet food', 'soup', 'beer', 'fromage blanc', ..."
79997,1546,9168204,"['cake', 'french fries', 'cooking oil', 'muffi..."
79998,7242,5054623,"['fromage blanc', 'napkins', 'sandwich', 'hone..."


In [17]:
basket['invoice_id'].nunique()

79749

In [19]:
aux = pd.DataFrame(basket.groupby('invoice_id').count())

In [20]:
aux[aux['list_of_goods'] > 1]

,customer_id,list_of_goods
invoice_id,,
52512,2,2
60470,2,2
60724,2,2
91497,2,2
135810,2,2
...,...,...
11700808,2,2
11708822,2,2
11724307,2,2


In [22]:
basket[basket['invoice_id'] == 52512]['list_of_goods']

22538    ['tomato juice', 'cake', 'grated cheese', 'cho...
22539    ['tomato juice', 'cake', 'grated cheese', 'cho...
Name: list_of_goods, dtype: object